In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport notes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from notes import Notes
from pprint import pprint
import math
import constants as const

### Load phrase base

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')

### Draw notations

In [43]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [44]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=5, 
                        active_bits=8, 
                        bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


[(8, 0), (0, 1), (13, 2), (24, 3), (18, 4)]


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [4]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

### Learn model

In [3]:
def feed_phrase_batch(cxnn: ContextNN, feeder: PhraseFeeder, abc_notes: Notes, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200)
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = abc_notes.phrase_chord(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)

def feed_n_batches(cxnn: ContextNN, feeder: PhraseFeeder, abc_notes: Notes, n=10, batch_size=200):
    for i in range(n):
        feed_phrase_batch(cxnn, feeder, abc_notes, count=batch_size)
        print(f'batch {i+1}/{n}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def learn_cycle(cxnn: ContextNN, 
                feeder: PhraseFeeder, 
                abc_notes: Notes,
                batch_size=200,
                n_accumulate_batches=30,
                n_consolidate_batches=50,
                reduce_min_component=0.2,
                reduce_min_activations=100):
    print('accumulating clusters...')
    cxnn.state = const.STATE_ACCUMULATE
    feed_n_batches(cxnn, feeder, abc_notes, n=n_accumulate_batches, batch_size=batch_size)
    
    print('\nconsolidating clusters...')
    cxnn.state = const.STATE_CONSOLIDATE
    feed_n_batches(cxnn, feeder, abc_notes, n=n_consolidate_batches, batch_size=batch_size)
    
    print('\nreducing clusters...')
    cxnn.reduce_clusters(min_component=reduce_min_component, 
                         min_activations=reduce_min_activations,
                         trim=True,
                         remain_part=0.3,
                         clear_stats=True,
                         consolidate=True,
                         amnesty=True)
    print(f'clusters: {cxnn.cluster_count()}')
    

In [4]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=5, 
                        active_bits=8, 
                        bit_count=255)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=255,
                            output_bit_count=8,
                            watch_point_count=100,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4)

In [13]:
%%time
for _ in range(10):
    learn_cycle(cxnn, 
                feeder, 
                abc_notes,
                batch_size=200,
                n_accumulate_batches=30,
                n_consolidate_batches=30,
                reduce_min_component=0.2,
                reduce_min_activations=150)

accumulating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batch 14/30
batch 15/30
batch 16/30
batch 17/30
batch 18/30
batch 19/30
batch 20/30
batch 21/30
batch 22/30
batch 23/30
batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 72898

consolidating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batch 14/30
batch 15/30
batch 16/30
batch 17/30
batch 18/30
batch 19/30
batch 20/30
batch 21/30
batch 22/30
batch 23/30
batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 72898

reducing clusters...
clusters: 30531
accumulating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batc

clusters: 109336
Wall time: 25min 54s


### Model stats

In [12]:
print(cxnn.cluster_count())
print(cxnn.point_stats())
marks

14969
[(2, 298), (5, 93), (4, 150), (4, 86), (5, 150), (5, 79), (4, 26), (6, 197), (0, 9), (7, 111), (6, 232), (5, 45), (6, 132), (5, 323), (0, 208), (0, 139), (5, 52), (1, 48), (3, 201), (2, 164), (7, 27), (4, 152), (6, 228), (4, 131), (0, 109), (5, 353), (7, 165), (0, 185), (0, 86), (0, 131), (2, 299), (2, 296), (0, 42), (5, 212), (3, 198), (2, 297), (0, 35), (0, 120), (7, 60), (2, 238), (2, 226), (0, 24), (6, 183), (6, 283), (5, 23), (5, 112), (5, 106), (6, 129), (4, 155), (5, 73), (0, 61), (2, 175), (2, 31), (5, 98), (5, 89), (7, 258), (0, 187), (4, 56), (2, 332), (2, 252), (1, 127), (0, 205), (6, 208), (7, 154), (4, 28), (1, 126), (1, 145), (4, 110), (3, 290), (3, 214), (7, 322), (7, 72), (5, 53), (0, 89), (6, 351), (1, 100), (1, 86), (0, 208), (1, 66), (6, 208), (5, 83), (6, 243), (1, 93), (3, 89), (4, 50), (7, 69), (4, 195), (4, 179), (4, 118), (3, 223), (0, 213), (7, 69), (7, 190), (0, 99), (5, 255), (0, 117), (2, 277), (7, 173), (1, 64), (4, 48)]


{(0, 2, 4, 6): 'pol',
 (0, 2, 5, 7): 'epo',
 (0, 3, 5, 6): 'jbo',
 (1, 2, 3, 4): 'blg',
 (1, 2, 6, 7): 'eng',
 (1, 3, 6, 7): 'bel',
 (2, 3, 5, 7): 'rus',
 (4, 5, 6, 7): 'ukr'}

In [65]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

17242
{4: 7681, 5: 6497, 6: 2295, 7: 599, 8: 154, 9: 15, 10: 1}
{0: 17242}


In [11]:
cxnn.cluster_consolidated_stats()
    

{1: 14969}

In [25]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = dict_value(cxnn.watch_points, n=0)
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

pprint(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(tuple(sorted(cluster.bits)))
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

{}
0
[]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
(36, 167, 199, 239)
(8, 13, 20, 26, 29, 36, 40, 59, 63, 77, 87, 103, 104, 105, 110, 125, 127, 133, 161, 167, 168, 173, 191, 197, 199, 212, 233, 237, 239, 243, 253, 254)
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0]
319 319


In [131]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

if cluster_with_stats:
    print(cluster_with_stats.stats, cluster_with_stats.consolidated)
else:
    print('nothing to print')
    
    

nothing to print


### Save-load model

In [7]:
import pickle

def save_cxnn(cxnn: ContextNN, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(cxnn, f)

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn


In [8]:
save_cxnn(cxnn, './data/cxnn_langs.pickle')

In [10]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')